In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import pymongo
import re
import pymysql
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()
import time

In [2]:
# getting gkey from the config file
from config import gkey

# getting the info for sql bench
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
# Creating engine to mysql
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [ ]:
#results[8]['photos'][0]['photo_reference']

In [4]:
def generate_results_dataframe(results):
    results_list = []

    for r in results:
        test = True

        # Initialize variables
        business_status = None
        lat = None
        lng = None
        icon = None,
        name = None
        place_id = None
        price_level = None
        photo_reference = None
        rating = None
        place_types = None
        user_ratings_total = None
        address = None

        try:
            if 'business_status' in r.keys(): business_status = r['business_status']
        except: pass

        try:
            if 'lat' in r['geometry']['location'].keys(): lat = r['geometry']['location']['lat']
        except: pass
        try:
            if 'lng' in r['geometry']['location'].keys(): lng = r['geometry']['location']['lng']
        except: pass

        try:
            if 'icon' in r.keys(): icon = r['icon']
        except: pass

        try:
            if 'name' in r.keys(): name = r['name']
        except: pass

        try:
            if 'place_id' in r.keys(): place_id = r['place_id']
        except: pass

        try:
            if 'price_level' in r.keys(): price_level = r['price_level']
        except: pass

        try:
            if 'photos' in r.keys(): photo_reference = r['photos'][0]['photo_reference']
        except: pass

        try:
            if 'rating' in r.keys(): rating = r['rating']
        except: pass

        try:
            if 'types' in r.keys(): place_types = r['types']
        except: pass

        try:
            if 'user_ratings_total' in r.keys(): user_ratings_total = r['user_ratings_total']
        except: pass

        try:
            if 'vicinity' in r.keys(): address = r['vicinity']
        except: pass

        result_dict = {
            'business_status': business_status,
            'lat': lat,
            'lng': lng,
            'icon': icon,
            'name': name,
            'place_id': place_id,
            'price_level': price_level,
            'photo_reference': photo_reference,
            'rating': rating,
            'place_types': place_types,
            'user_ratings_total': user_ratings_total,
            'address': address
        }

        results_list.append(result_dict)

    results_df = pd.DataFrame(results_list)
    results_df['place_types'] = results_df['place_types'].astype(str)
    
    return results_df

In [6]:
def load_into_database(response):

    while('results' in response.keys() and len(response['results']) > 0):   

        results = response['results']

        # call generate_results_dataframe function and pass the API response results to it
        # this will return a DataFrame that contains the data returned by the API
        results_df = generate_results_dataframe(results)

        # post to SQL
        # PLEASE DO NOT WRITE TO THE restraurants TABLE!!!
        # Write to your own table by place a suffix behind restaurants
        results_df.to_sql('restaurants_test', con=conn, if_exists='append', index=False)

        print(f'--- {len(results_df)} record(s) loaded to the database --- ')

        if 'next_page_token' in response:      
            pagetoken = response['next_page_token']

            # sleeping for the "short delay" 
            time.sleep(10)

            base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
            params = {
                "pagetoken": pagetoken,
                "key": gkey,
            }

            response = requests.get(base_url, params=params).json()

        else:
            break

In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

san_juan = ['18.466333, -66.105721']
ponce = ['18.01108, -66.61406']
rincon = ['18.340216, -67.250015']
fajardo = ['18.325787, -65.652382']
arecibo = ['18.47245, -66.71573']
humacao = ['18.148750, -65.819099']
mayaguez = ['18.201349, -67.139488']
# spanish, puertorican, middle eastern, chinese, fast food, bar restaurant

cities = [san_juan, ponce, rincon, fajardo, arecibo, humacao, mayaguez]

for city in cities:
    print(f'Processing {city}')

    params = {
        "location": city,  # coords
        "radius": 8000,
        "types": "restaurant",
        "key": gkey,
        "keyword": ""
    }

    # Initial API call
    response = requests.get(base_url, params=params).json()
    
    load_into_database(response)

Processing ['18.466333, -66.105721']
--- 20 record(s) loaded to the database --- 
--- 20 record(s) loaded to the database --- 
--- 20 record(s) loaded to the database --- 
Processing ['18.01108, -66.61406']
--- 20 record(s) loaded to the database --- 
--- 20 record(s) loaded to the database --- 
--- 20 record(s) loaded to the database --- 
Processing ['18.340216, -67.250015']
--- 20 record(s) loaded to the database --- 
--- 20 record(s) loaded to the database --- 
--- 20 record(s) loaded to the database --- 
Processing ['18.325787, -65.652382']
--- 20 record(s) loaded to the database --- 
--- 20 record(s) loaded to the database --- 
--- 20 record(s) loaded to the database --- 
Processing ['18.47245, -66.71573']
--- 20 record(s) loaded to the database --- 
--- 20 record(s) loaded to the database --- 
--- 20 record(s) loaded to the database --- 
Processing ['18.148750, -65.819099']
--- 20 record(s) loaded to the database --- 
--- 20 record(s) loaded to the database --- 
--- 20 record(s) 